In [ ]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import fcluster

In [ ]:
path = 'G:\\Dropbox (Vetsigian lab)\\Vetsigian lab Team Folder\\Ye\\Genomics\\'
# path = 'E:\\CMT_project\\CMT project\\'
genomic_table_coelicolor = pd.read_csv(path + 'genomic_table_coelicolor.csv')
path = 'G:\\Dropbox (Vetsigian lab)\\Vetsigian lab Team Folder\\Ye\Genomics\\Community mutants\\antiSMASH_Streptomyces_coelicolor\\'
# path = 'E:\\CMT_project\\CMT project\\'
filename = 'index.html'
cluster_info = pd.read_html(path+filename)[0].drop(0)

In [ ]:
# Import filled variance files of each population

filepath = 'G:\\Dropbox (Vetsigian lab)\\Vetsigian lab Team Folder\\Ye\\Genomics\\Community mutants\\'
folders = ['PopA', 'PopB', 'PopC', 'PopD', 'PopE', 'PopF', 'PopG', 'PopH']

vcfA = pd.read_csv(filepath + folders[0] + '\\vcfA_filled.csv')
# vcfB = pd.read_csv(filepath + folders[1] + '\\vcfB_filled.csv')
vcfC = pd.read_csv(filepath + folders[2] + '\\vcfC_filled.csv')
vcfD = pd.read_csv(filepath + folders[3] + '\\vcfD_filled.csv')
# vcfE = pd.read_csv(filepath + folders[4] + '\\vcfE_filled.csv')
# vcfF = pd.read_csv(filepath + folders[5] + '\\vcfF_filled.csv')
# vcfG = pd.read_csv(filepath + folders[6] + '\\vcfG_filled.csv')
# vcfH = pd.read_csv(filepath + folders[7] + '\\vcfH_filled.csv')


In [ ]:
vcfA.head(5)

In [ ]:
def plot_variance_abundance_along_timepoints(vcfs,time_points, pop_lab):
    '''
    Inputs:
    vcfs = variance files of teh query population(e.g. PopA) 
    timepoints 
    pop_lab = population labels (e.g. 'A')
    
    Outputs:
    plot of variance abundance vs. timepoints
    '''
    VF_cols = vcfs.columns[8::2]
    for i in range(0, len(vcfs)):
        plt.plot(time_points, vcfs.iloc[i][VF_cols])
    plt.xlabel('Community Generations')
    plt.ylabel('Variance Frequency')
    plt.title('Population ' + pop_lab)

In [ ]:
# 1. PopA variance profile visualization
colnames  = vcfA.columns
timepoints = [int(vf[0]) for vf in colnames[8::2]]
coverage = colnames[9::2]
# Find variances covered through all tested generations/timepoints
mask = np.sum(vcfA[coverage] > 20, axis = 1) == len(timepoints)
vcfA_full = vcfA.loc[mask]
plot_variance_abundance_along_timepoints(vcfA_full,timepoints, 'A')


In [ ]:
# 2. PopC variance profile visualization
colnames  = vcfC.columns
timepoints = [int(vf[0]) for vf in colnames[8::2]]
coverage = colnames[9::2]
# Find variances covered through all tested generations/timepoints
mask = np.sum(vcfC[coverage] > 20, axis = 1) == len(timepoints)
vcfC_full = vcfC.loc[mask]
plot_variance_abundance_along_timepoints(vcfC_full,timepoints, 'C')


In [ ]:
vcfC_full.iloc[150:160][colnames[8:]]

In [ ]:
# 3. PopD variance profile visualization
colnames  = vcfD.columns
timepoints = [int(vf[0]) for vf in colnames[8::2]]
coverage = colnames[9::2]
# Find variances covered at least 20x through all tested generations/timepoints
mask = np.sum(vcfD[coverage] > 20, axis = 1) == len(timepoints)
vcfD_full = vcfD.loc[mask]
plot_variance_abundance_along_timepoints(vcfD_full,timepoints, 'D')

In [ ]:
vf = colnames[8::2]
msk1 = np.sum(vcfD_full[vf[0:4]] > 0, axis = 1) == 4
D1 = vcfD_full.loc[msk1]
plot_variance_abundance_along_timepoints(D1,timepoints, 'D1')
D1_loc1 = D1['Minimum']
D1_loc2 = D1['Maximum']
plt.figure()
plt.plot(D1_loc1, D1_loc2, 'bo')
plt.xticks(np.arange(0, 9000000, 10**6), rotation = 90)
plt.yticks(np.arange(0, 9000000, 10**6))


In [ ]:
down123 = np.sum(vcfD_full[vf[0:3]] == 0, axis = 1) == 3
down678 = np.sum(vcfD_full[vf[4:]] == 0, axis = 1) == 3
up4 = vcfD_full[vf[3]] > 0
print(len(down123), len(down678), len(up4), len(vcfD_full))

In [ ]:
msk_4up_only = np.sum(pd.concat([down123, down678, up4], axis =1), axis = 1) == 3
D_4up_only = vcfD_full.loc[msk_4up_only]
plot_variance_abundance_along_timepoints(D_4up_only,timepoints, 'D_4up_only')
loc1 = D_4up_only['Minimum']
loc2 = D_4up_only['Maximum']
plt.figure()
plt.plot(loc1, loc2, 'bo')
plt.xticks(np.arange(0, 9000000, 10**6), rotation = 90)
plt.yticks(np.arange(0, 9000000, 10**6))

In [ ]:
down1234 = np.sum(vcfD_full[vf[0:4]] == 0, axis = 1) == 4
down78 = np.sum(vcfD_full[vf[5:7]] == 0, axis = 1) == 2
up6 = vcfD_full[vf[4]] > 0

In [ ]:
msk_6up_only = np.sum(pd.concat([down1234, down78, up6], axis =1), axis = 1) == 3
D_6up_only = vcfD_full.loc[msk_6up_only]
plot_variance_abundance_along_timepoints(D_6up_only,timepoints, 'D_6up_only')
loc1 = D_6up_only['Minimum']
loc2 = D_6up_only['Maximum']
plt.figure()
plt.plot(loc1, loc2, 'bo')
plt.xticks(np.arange(0, 9000000, 10**6), rotation = 90)
plt.yticks(np.arange(0, 9000000, 10**6))

In [ ]:
D_6up_only

In [ ]:
# 3. Build variance correlation for each population
colA_vf = vcfA.columns[8::2]
dist_mat = np.array(1 - vcfA[colA_vf].T.corr())

m = dist_mat.shape[0]
r = np.arange(m)
mask = r[:, None]<r
dist_mat2 = np.nan_to_num(dist_mat[mask])

dist_mat2[dist_mat2 < 0] = 0

methods = ['single', 'complete', 'average', 'ward', 'weighted', 'centroid']
Z = dict()
c = dict()
coph_dists = dict()
plt.figure()

for med in methods:
    Z[med] = linkage(dist_mat2, med)
    c[med], coph_dists[med] = cophenet(Z[med], dist_mat2)
    plt.bar(med, c[med], color = 'b')

In [ ]:
Z = linkage(dist_mat2, 'average')

In [ ]:
def dendrogram_profile(profile_link):
    # calculate full dendrogram
    # save_path = 'E:\\CMT_project\\CMT project\\clustering_common_variances.png'
#     plt.figure()
    plt.figure(1, figsize=(12, 6))
    plt.title('Hierarchical Clustering of variances', size = 16)
    plt.xlabel('variance index', size = 14)
    plt.ylabel('distance', size = 14)
    R = dendrogram(profile_link,
#                    truncate_mode='lastp',  # show only the last p merged clusters
#                    p=50,  # show only the last p merged clusters
                   show_leaf_counts=False,  # otherwise numbers in brackets are counts
                   leaf_rotation=90.,
                   leaf_font_size=12,
                   show_contracted=True,  # to get a distribution impression in truncated branches
                   color_threshold = 0.85,
#                    orientation = 'left'
                  )
#     save_path = 'E:\\CMT_project\\CMT project\\hierarchical_clustering_of_common_variances.png'
#     plt.savefig(save_path)
    plt.show()

In [ ]:
dendrogram_profile(Z)

In [ ]:
def find_cluster_for_SNPs(cluster_info, snp2geno_cat):
    '''
    Inputs:
    cluster_info = the cluster information dataframe from the antiSMASH website
    snp2geno_cat = df of SNPs contribute to the categorized dynamics of the variance frequency 
            of the communities along time,generated by find_genetic_region_for_SNPs()
    Outputs:
    The cluster info the genes belong to if found
    '''
    cluster_range = cluster_info.apply(lambda row: range(int(row['From']), int(row['To'])+1), axis = 1)
    SNP_ranges = snp2geno_cat.reset_index().apply(lambda row: range(int(row['Minimum']), int(row['Maximum']) + 1), axis = 1)
    
    colnames_cluster = cluster_info.columns
    colnames_genes = snp2geno_cat.columns
    
    cluster_slice = pd.DataFrame(columns= colnames_cluster)
    gene_slice = pd.DataFrame(columns = colnames_genes)     
    
    count1 = 0
    for snp_r in SNP_ranges:
        snp_r = set(snp_r)
        count2 = 0
        for clk in cluster_range:
            ovp = snp_r.intersection(clk)
            if bool(ovp):
                temp1 = snp2geno_cat.iloc[count1]
                gene_slice = gene_slice.append(temp1)
                temp2 = cluster_info.iloc[count2]
                cluster_slice = cluster_slice.append(temp2)
            count2 = count2 + 1
        count1 = count1 + 1
        
    # Concat the two dfs
    df1 = gene_slice.reset_index().drop("index", axis = 1)    
    df2 = cluster_slice.reset_index().drop("index", axis = 1)[["Cluster", "Type"]]
    gene2cluster = pd.concat([df1, df2], axis = 1)

    return gene2cluster

In [ ]:
max_d = 0.85
clusters = fcluster(Z, max_d, criterion='distance')
clustered_profile = dict()
for cl in set(clusters):
    clustered_profile[cl] = vcfA.loc[clusters == cl]
    df_cluster = find_cluster_for_SNPs(cluster_info, clustered_profile[cl])
    cluster = df_cluster['Cluster'].values
    Type = df_cluster['Type'].values
    print(str(cl) + ':' + cluster + ',' + Type)

In [ ]:
# from rpy2.robjects import r, pandas2ri
# from rpy2.robjects.packages import importr
# pandas2ri.activate()
# base = importr("base")
# pvclust = importr("pvclust")
# data = robjects.DataFrame.from_csvfile(filepath + folders[0] + '\\vcfA_filled.csv')
# data

In [ ]:
# cols = robjects.IntVector(np.arange(9, 21, 2))
# cols
# subset = data.rx(True, cols)
# is_numeric(subset)
# # result = pvclust.pvclust(data, method_hclust = "average", method_dist = "correlation", nboot = 1000)

In [ ]:
def variance_profile_per_population(vcfs_pop, var_T1):
    cols = vcfs_pop.columns
    cols1 = cols[0:7]
    cols2 = cols[7:]
    profile = pd.DataFrame(columns = cols)
    for ind in range(len(var_T1)):
        vari = var_T1.iloc[ind]
        msk = np.sum(vcfs_pop[['Name', 'Minimum', 'Maximum']] == vari, axis = 1) == 3
        if sum(msk) > 1:
            temp1 = vcfs_pop.loc[msk,cols1].iloc[0]        
            temp2 = vcfs_pop.loc[msk,cols2].sum()
            temp = pd.concat([temp1, temp2])
            profile = profile.append(temp, ignore_index = True)
        else:
            profile = profile.append(vcfs_pop.loc[msk, cols], ignore_index = True)
    return profile